In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('/Users/juliankimura/Desktop/deepomics')
import deepomics.neuralnetwork as nn
from deepomics import learn, utils
from models import MNIST_model

import theano
import theano.tensor as T
from lasagne import layers

np.random.seed(247)   # for reproducibility

In [3]:
def load_dataset(data_path):
    import gzip
    def load_mnist_images(filename):
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
        data = data.reshape(-1, 28*28)
        return data / np.float32(256)

    def load_mnist_labels(filename):
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)
        return data

    # We can now download and read the training and test set images and labels.
    X_train = load_mnist_images(os.path.join(data_path,'train-images-idx3-ubyte.gz')).reshape([-1, 1, 28, 28])
    y_train_index = load_mnist_labels(os.path.join(data_path,'train-labels-idx1-ubyte.gz'))
    X_test = load_mnist_images(os.path.join(data_path,'t10k-images-idx3-ubyte.gz')).reshape([-1, 1, 28, 28])
    y_test_index = load_mnist_labels(os.path.join(data_path,'t10k-labels-idx1-ubyte.gz'))

    y_train = np.zeros((y_train_index.shape[0], 10)).astype(np.float32)
    for i in range(y_train.shape[0]):
        y_train[i,y_train_index[i]] = 1
    y_test = np.zeros((y_test_index.shape[0], 10)).astype(np.float32)
    for i in range(y_test.shape[0]):
        y_test[i,y_test_index[i]] = 1
    
    # We reserve the last 10000 training examples for validation.
    X_train, X_val = X_train[:-10000], X_train[-10000:]
    y_train, y_val = y_train[:-10000], y_train[-10000:]

    train = (X_train, y_train)
    valid = (X_val, y_val)
    test = (X_test, y_test)

    return train, valid, test

data_path = '/Users/juliankimura/Desktop/data/MNIST'
train, valid, test = load_dataset(data_path)

In [4]:
# build model
input_shape = list(train[0].shape)
input_shape[0] = None
output_shape = train[1].shape
network, placeholders, optimization = MNIST_model.model(input_shape, output_shape)

In [5]:
# build neural network class
nnmodel = nn.NeuralNet(network, placeholders)
nnmodel.inspect_layers()

# set output file paths
output_name = 'MNIST_vae'
results_path = utils.make_directory(data_path, 'Results')
results_path = utils.make_directory(results_path, output_name)
file_path = os.path.join(results_path, output_name)
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', file_path=file_path)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: 
shape:(None, 1, 28, 28)
layer2: 
shape:(None, 16, 28, 28)
parameters: W
layer3: 
shape:(None, 16, 28, 28)
parameters: beta, gamma, mean, inv_std
layer4: 
shape:(None, 16, 28, 28)
layer5: 
shape:(None, 16, 28, 28)
layer6: 
shape:(None, 16, 14, 14)
layer7: 
shape:(None, 32, 14, 14)
parameters: W
layer8: 
shape:(None, 32, 14, 14)
parameters: beta, gamma, mean, inv_std
layer9: 
shape:(None, 32, 14, 14)
layer10: 
shape:(None, 32, 14, 14)
layer11: 
shape:(None, 32, 7, 7)
layer12: 
shape:(None, 512)
parameters: W
layer13: 
shape:(None, 512)
parameters: beta, gamma, mean, inv_std
layer14: 
shape:(None, 512)
layer15: 
shape:(None, 10)
parameters: W
layer16: 
shape:(None, 10)
parameters: b
layer17: 
shape:(None, 10)
----------------------------------------------------------------------------
compiling model


In [6]:
# train model
learn.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=500, patience=10, verbose=1)

Epoch 1 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.11105 -- accuracy=96.67%  
  valid loss:		0.11625
  valid accuracy:	0.99321+/-0.00453
  valid auc-roc:	0.99949+/-0.00045
  valid auc-pr:		0.99699+/-0.00207
saving model parameters to: /Users/juliankimura/Desktop/data/MNIST/Results/MNIST_vae/MNIST_vae_best.pickle
Epoch 2 out of 500 
[===                           ] 10.8% -- time=117s -- loss=0.04190 -- accuracy=98.72%  

KeyboardInterrupt: 

In [ ]:
# load best model --> lowest cross-validation error
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(test, name="test", batch_size=100)